In [1]:
!apt-get update -qq
!apt-get install -y libopenmpi-dev openmpi-bin openmpi-common
!pip install mpi4py

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from mpi4py import MPI

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Model definition (same for all nodes)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile once (before training loop)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Load and prepare data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[..., tf.newaxis] / 255.0  # Add channel dim
x_test = x_test[..., tf.newaxis] / 255.0

def train(model, x_train, y_train):
    # Data splitting
    n = len(x_train)
    chunk_size = n // size
    start = rank * chunk_size
    end = (rank + 1) * chunk_size if rank != size-1 else n

    # Local training
    model.fit(x_train[start:end], y_train[start:end],
              epochs=1, batch_size=32, verbose=0)

    # Weight synchronization
    weights = model.get_weights()
    weights = comm.allreduce(weights, op=MPI.SUM)
    model.set_weights([w / size for w in weights])

# Training loop
epochs = 5
for epoch in range(epochs):
    train(model, x_train, y_train)

    # Rank 0 handles evaluation
    if rank == 0:
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
        print(f"Epoch {epoch+1}: Test accuracy = {test_acc:.4f}")


RuntimeError: cannot load MPI library
Could not find module 'C:\Users\Adesh Punde\AppData\Roaming\Python\DLLs' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Users\Adesh Punde\AppData\Roaming\Python\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\Adesh Punde\AppData\Local\Programs\Python\Python312\DLLs\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\Adesh Punde\AppData\Local\Programs\Python\Python312\DLLs\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'c:\Users\Adesh Punde\AppData\Local\Programs\Python\Python312\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could 